In [22]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [28]:
!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
#!python3 -m pip install paddlepaddle-gpu==2.0.2 -f https://paddlepaddle.org.cn/whl/stable/noavx.html
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

######Dont install#!python3 -m pip install paddlepaddle-gpu
!python -m pip install --force-reinstall paddlepaddle==2.5

!pip install "paddleocr>=2.0.1"
#!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

### Install first one if code doea not work install others
!pip install protobuf==3.20.*
#!pip install -U layoutparser-0.0.0-py3-none-any.whl
#!wget https://www.openssl.org/source/openssl-1.1.1o.tar.gz
#!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
#!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubun

Processing ./layoutparser-0.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=264d7892dbe6d33981e9b3bb7670cd9d060fc9d77bf408fcd4a610ec2ed97f80
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath


In [1]:
from pdf2image import convert_from_path

In [2]:
#from google.colab import files
#uploaded = files.upload()

In [3]:
images = convert_from_path('/content/data/pI - 2023-08-31T064223.490.pdf')

In [4]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [5]:
for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg', 'JPEG')

In [6]:
import cv2
import layoutparser as lp
image = cv2.imread("/content/pages/page0.jpg")

image = image[..., ::-1]

# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.5,
                                #label_map={0:"table"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library
# detect
layout = model.detect(image)

download https://paddle-model-ecology.bj.bcebos.com/model/layout-parser/ppyolov2_r50vd_dcn_365e_publaynet.tar to /root/.paddledet/inference_model/ppyolov2_r50vd_dcn_365e_publaynet/ppyolov2_r50vd_dcn_365e_publaynet_infer/ppyolov2_r50vd_dcn_365e_publaynet.tar


100%|██████████| 221M/221M [00:06<00:00, 36.6MiB/s]


In [7]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=71.61138916015625, y_1=14.5443115234375, x_2=1586.759521484375, y_2=2162.3544921875), text=None, id=None, type=Figure, parent=None, next=None, score=0.9283877015113831), TextBlock(block=Rectangle(x_1=79.14984130859375, y_1=1649.6763916015625, x_2=1073.70556640625, y_2=1679.0364990234375), text=None, id=None, type=Text, parent=None, next=None, score=0.6794461011886597)], page_data={})

In [8]:
x_1=0
y_1=0
x_2=0
y_2=0
for l in layout:
  #print(l)
  if l.type == 'Figure':
    x_1 = int(l.block.x_1)
    print(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)

    break

71.61139


In [9]:
im = cv2.imread('/content/pages/page0.jpg')

In [10]:
cv2.imwrite('ext_im.jpg', im[y_1:y_2,x_1:x_2])

True

In [11]:
from paddleocr import PaddleOCR, draw_ocr

In [12]:
ocr = PaddleOCR(lang='en')
image_path = '/content/ext_im.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)[0]

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:00<00:00, 7.08MiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:00<00:00, 15.1MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:00<00:00, 4.76MiB/s]

[2023/09/22 16:24:28] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

[2023/09/22 16:24:29] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2023/09/22 16:24:31] ppocr DEBUG: dt_boxes num : 284, elapsed : 1.159461498260498
[2023/09/22 16:25:20] ppocr DEBUG: rec_res num  : 284, elapsed : 49.4906165599823


In [13]:
print(output)

[[[[613.0, 31.0], [755.0, 31.0], [755.0, 56.0], [613.0, 56.0]], ('Tax Invoice', 0.9908809661865234)], [[[162.0, 54.0], [201.0, 54.0], [201.0, 78.0], [162.0, 78.0]], ('SRI', 0.9911784529685974)], [[[1068.0, 100.0], [1232.0, 106.0], [1230.0, 159.0], [1066.0, 154.0]], ('fssal', 0.8870811462402344)], [[[58.0, 150.0], [293.0, 145.0], [293.0, 183.0], [59.0, 188.0]], ('BHAGYALAKSHM', 0.9222421646118164)], [[[1051.0, 168.0], [1432.0, 170.0], [1431.0, 201.0], [1051.0, 199.0]], ('FGL:ILR ECL (AL452/90-91', 0.9146592020988464)], [[[132.0, 250.0], [230.0, 255.0], [229.0, 280.0], [130.0, 275.0]], ('Since 1972', 0.955083966255188)], [[[7.0, 286.0], [129.0, 286.0], [129.0, 309.0], [7.0, 309.0]], ('Consignee', 0.998616099357605)], [[[183.0, 282.0], [417.0, 282.0], [417.0, 311.0], [183.0, 311.0]], ('Party Id:DST-0120', 0.96061772108078)], [[[676.0, 280.0], [778.0, 280.0], [778.0, 311.0], [676.0, 311.0]], ('Po No. :', 0.929901659488678)], [[[978.0, 284.0], [1109.0, 284.0], [1109.0, 309.0], [978.0, 309.0

In [14]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]

In [15]:
image_boxes = image_cv.copy()

In [16]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [17]:
cv2.imwrite('detections.jpg', image_boxes)

True

In [18]:
im = image_cv.copy()

In [19]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)


In [20]:
cv2.imwrite('horiz_vert.jpg',im)

True

In [23]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [24]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[  0   1   3   5   6  13  14  20  22  25  33  36  42  49  60  66  74  77
  89  95 103 113 121 129 136 139 146 150 159 169 177 184 188 192 194 196
 198 200 201 203 204 212 216 226 229 235 243 253 256 268 276 277 281]


In [25]:
im_nms = image_cv.copy()

In [26]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [27]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [28]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [29]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[ 22  36  49  60  74  89 159 204 206 228 229 246 268]


In [30]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)


In [31]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [ ]:
out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

In [33]:

unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

[679, 0, 849, 2148]
[1184, 0, 1242, 2148]
[942, 0, 1028, 2148]
[1386, 0, 1492, 2148]
[23, 0, 43, 2148]
[1096, 0, 1118, 2148]
[857, 0, 877, 2148]
[115, 0, 237, 2148]
[523, 0, 706, 2148]
[239, 0, 358, 2148]
[476, 0, 510, 2148]
[1015, 0, 1100, 2148]
[1238, 0, 1296, 2148]


In [34]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

[ 4  7  9 10  8  0  6  2 11  5  1 12  3]


In [35]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [36]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [37]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [38]:
indices = [index for index, item in enumerate(out_array) if "Description of Goods" in item]

In [39]:
indices

[]

In [40]:
indices_start = [index for index, item in enumerate(out_array) if "PRODUCT DESCRIPTION" in item or "Description of Goods" in item]
indices_end = [index for index, item in enumerate(out_array) if "Total" in item]
#indices_end = np.where(out_array == "Total")[0]

In [41]:
indices_start[0]

10

In [42]:
indices_end[0]

32

In [43]:
out_array1=np.array(out_array[indices_start[0]:indices_end[0]+1])

In [44]:
df1 = pd.DataFrame(out_array1)

In [45]:
df1.to_csv('sample1.csv',index = False)

In [46]:
#out_array

In [47]:
indices_start1 = [index for index, item in enumerate(out_array) if "HSN/SAC" in item]
indices_end1 = [index for index, item in enumerate(out_array) if "Page 1/2" in item or "Page 2/2" in item or "Goods Once sold" in item]

In [48]:
indices_end1

[52]

In [49]:
if len(indices_start1) >0 and len(indices_end1)>0:
  out_array2=np.array(out_array[indices_start1[0]:indices_end1[0]])
  df2 = pd.DataFrame(out_array2)
  df2.to_csv('sample2.csv',index = False)